In [23]:
import pandas as pd
import numpy as np

csv_file_path = '../DATA/RAW/dirty_cafe_sales.csv'

csv_file = pd.read_csv(csv_file_path)

# Error Summary:
#   Transaction ID: {}
#   Item: {'UNKNOWN': 344, nan: 333, 'ERROR': 292}
#   Quantity: {'UNKNOWN': 171, 'ERROR': 170, nan: 138}
#   Price Per Unit: {'ERROR': 190, nan: 179, 'UNKNOWN': 164}
#   Total Spent: {nan: 173, 'UNKNOWN': 165, 'ERROR': 164}
#   Payment Method: {nan: 2579}
#   Location: {nan: 3265, 'ERROR': 358, 'UNKNOWN': 338}
#   Transaction Date: {nan: 159, 'UNKNOWN': 159, 'ERROR': 142}

# Turn every Error and missing value into nan
csv_file.replace({'ERROR': pd.NA, 'UNKNOWN': pd.NA,'': pd.NA, 'nan': pd.NA}, inplace=True)


In [ ]:
#Drop all rows where 'Item' is missing

csv_file.dropna(subset=['Item'], inplace=True)

In [25]:
#Type change for numerical columns

num_cols = ['Quantity', 'Price Per Unit', 'Total Spent']
csv_file[num_cols] = csv_file[num_cols].apply(pd.to_numeric, errors='coerce')


In [26]:
# Fix Incoherence in Quantity, Price Per Unit, Total Spent

csv_file.dropna(subset=num_cols, thresh=2, inplace=True)


mask_all = csv_file[num_cols].notna().all(axis=1)
expected = csv_file.loc[mask_all, 'Quantity'] * csv_file.loc[mask_all, 'Price Per Unit']
mismatch_idx = csv_file.loc[mask_all].index[~np.isclose(csv_file.loc[mask_all, 'Total Spent'], expected, rtol=1e-5, atol=1e-8)]

csv_file.drop(index=mismatch_idx, inplace=True)

mask = (
    csv_file['Total Spent'].notna() &
    csv_file['Price Per Unit'].notna() &
    (csv_file['Price Per Unit'] != 0)
)

csv_file.loc[mask, 'Quantity'] = csv_file.loc[mask, 'Total Spent'] / csv_file.loc[mask, 'Price Per Unit']

mask = (
    csv_file['Total Spent'].notna() &
    csv_file['Quantity'].notna() &
    (csv_file['Quantity'] != 0)
)
csv_file.loc[mask, 'Price Per Unit'] = csv_file.loc[mask, 'Total Spent'] / csv_file.loc[mask, 'Quantity']
mask = (
    csv_file['Price Per Unit'].notna() &
    csv_file['Quantity'].notna()
)
csv_file.loc[mask, 'Total Spent'] = csv_file.loc[mask, 'Quantity'] * csv_file.loc[mask, 'Price Per Unit']


In [27]:
csv_file.to_csv('../DATA/PROCESSED/cleaned_cafe_sales.csv', index=False)